In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import sqlite3

%matplotlib inline
#bommovies dataframe this is bad data
bom_movies_df = pd.read_csv('../data/bom.movie_gross.csv.gz')
#imbd dataframe
con = sqlite3.connect('../data/im.db') 
cursor = con.cursor()
imdb_schema_df = pd.read_sql('''
SELECT 
    *
FROM
    sqlite_master
''', con)   
#imdb directors df
directors_df = pd.read_sql('''select * from directors''',con)
#imdb writers df
writers_df = pd.read_sql('''select * from writers''',con)
#imdb know for df
known_for_df = pd.read_sql('''select * from known_for''',con)
#imdb principals df
principals_df = pd.read_sql('''select * from principals''',con)
#imdb persons df
persons_df = pd.read_sql('''select * from persons''',con)
#imdb movie basics df
movie_basics_df = pd.read_sql('''select * from movie_basics''',con)
#imdb movie ratings df
movie_ratings_df = pd.read_sql('''select * from movie_ratings''',con)
#imdb movie akas df
movie_akas_df = pd.read_sql('''select * from movie_akas''',con)
#rotten tomatoes movies df this is bad data
rt_movies_df = pd.read_csv('../data/rt.movie_info.tsv.gz', 
                               sep='\t') # sep='\t' needs to be specified since is a .tsv file
#rotten tomatoes reviews df this is bad data
rt_reviews_df = pd.read_csv('../data/rt.reviews.tsv.gz', 
                                sep='\t', # sep='\t' needs to be specified since is a .tsv file
                                encoding='unicode_escape') # using default utf-8 encoding ran into issues with this file
# TheMovieDB movie information table
tmdb_movies_df = pd.read_csv('../data/tmdb.movies.csv.gz')
# The Numbers movie information table
tn_movies_df = pd.read_csv('../data/tn.movie_budgets.csv.gz')

In [2]:
tn_movies_df.info() #  create ROI metric

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [3]:
tmdb_movies_df = tmdb_movies_df.set_index('id') #setting indexs and dropping columns that are not relevant 

In [4]:
movie_basics_df = movie_basics_df.set_index('movie_id')

In [5]:
tmdb_movies_df = tmdb_movies_df.drop('original_language', axis=1)

In [6]:
tmdb_movies_df = tmdb_movies_df.drop('title', axis=1)

In [7]:
tmdb_movies_df = tmdb_movies_df.drop('Unnamed: 0', axis=1)

In [8]:
tmdb_movies_df = tmdb_movies_df.drop('vote_count', axis=1)

In [9]:
tmdb_movies_df = tmdb_movies_df.drop('popularity', axis=1)

In [10]:
tmdb_movies_df = tmdb_movies_df.drop('vote_average', axis=1)

In [11]:
movie_basics_df = movie_basics_df.drop('runtime_minutes', axis=1)

In [12]:
movie_basics_df = movie_basics_df.drop('original_title', axis=1)

In [13]:
tmdb_movies_df['release_date'] = pd.to_datetime(tmdb_movies_df['release_date'], format = '%Y%') 

In [14]:
tmdb_movies_df['year'] = tmdb_movies_df['release_date'].dt.year

In [15]:
tmdb_movies_df = tmdb_movies_df.drop('release_date', axis=1)

In [16]:
tmdb_movies_df = tmdb_movies_df.set_index('original_title')

In [17]:
movie_basics_df = movie_basics_df.set_index('primary_title')

In [18]:
tn_movies_df = tn_movies_df.set_index('movie')

In [19]:
tn_movies_df = tn_movies_df.drop('id', axis = 1)

In [20]:
genre_df = movie_basics_df.join(tmdb_movies_df, how = 'inner') #joining genre databases

In [21]:
final_df = genre_df.join(tn_movies_df, how='inner') #joining genre databases with the numbers database aka money

In [22]:
final_df = final_df.drop('year', axis=1) #dropping duplicate dates

In [23]:
final_df = final_df.drop('start_year', axis=1)

In [24]:
final_df = final_df[final_df.worldwide_gross != '$0'] #dropping worldwide gross values that equal 0

In [25]:
final_df = final_df.dropna() #dropping empty genre values

In [26]:
final_df = final_df.drop_duplicates() #dropping exact duplicates from df, still need to merge or drop duplicates that are not exact

In [27]:
final_df['genre_ids'] = final_df["genre_ids"].str.split("," , n = 1, expand = False)

In [28]:
final_df = final_df.drop('genres', axis=1) # dropping genres column and using genre_ids as that one seems to place the most relevant genre first

In [29]:
final_df['Genre'] = final_df.genre_ids.apply(lambda x: x[0])

In [30]:
final_df['Genre'].replace(regex=True, inplace=True, to_replace=r'[^0-9.\-]',
value=r'') #cleaning the new Genres tab

In [31]:
final_df = final_df.drop('genre_ids', axis =1)

In [32]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2974 entries, 10 Cloverfield Lane to xXx: Return of Xander Cage
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   release_date       2974 non-null   object
 1   production_budget  2974 non-null   object
 2   domestic_gross     2974 non-null   object
 3   worldwide_gross    2974 non-null   object
 4   Genre              2974 non-null   object
dtypes: object(5)
memory usage: 139.4+ KB


In [33]:
final_df['Action'] = final_df['Genre'].isin(['28'])

In [34]:
final_df['Adventure'] = final_df['Genre'].isin(['12'])

In [35]:
final_df['Animation'] = final_df['Genre'].isin(['16'])

In [36]:
final_df['Comedy'] = final_df['Genre'].isin(['35'])

In [37]:
final_df['Crime'] = final_df['Genre'].isin(['80'])

In [38]:
final_df['Documentary'] = final_df['Genre'].isin(['99'])

In [39]:
final_df['Drama'] = final_df['Genre'].isin(['18'])

In [40]:
final_df['Family'] = final_df['Genre'].isin(['10751'])

In [41]:
final_df['Fantasy'] = final_df['Genre'].isin(['14'])

In [42]:
final_df['History'] = final_df['Genre'].isin(['36'])

In [43]:
final_df['Horror'] = final_df['Genre'].isin(['27'])

In [44]:
final_df['Mystery'] = final_df['Genre'].isin(['9648'])

In [45]:
final_df['Music'] = final_df['Genre'].isin(['10402'])

In [46]:
final_df['Romance'] = final_df['Genre'].isin(['10749'])

In [47]:
final_df['SciFi'] = final_df['Genre'].isin(['878'])

In [48]:
final_df['TV_Movie'] = final_df['Genre'].isin(['10770'])

In [49]:
final_df['Thriller'] = final_df['Genre'].isin(['53'])

In [50]:
final_df['War'] = final_df['Genre'].isin(['10752'])

In [51]:
final_df['Western'] = final_df['Genre'].isin(['37'])

In [52]:
final_df['production_budget'].replace(regex=True, inplace=True, to_replace=r'[^0-9.\-]',
value=r'') #cleaning buget, domestic, and worldwide

In [53]:
final_df['domestic_gross'].replace(regex=True, inplace=True, to_replace=r'[^0-9.\-]',
value=r'') #cleaning buget, domestic, and worldwide

In [54]:
final_df['worldwide_gross'].replace(regex=True, inplace=True, to_replace=r'[^0-9.\-]',
value=r'') #cleaning buget, domestic, and worldwide

In [55]:
final_df = final_df.astype({'production_budget':'float'}) # converting objects to floats

In [56]:
final_df = final_df.astype({'domestic_gross':'float'})# converting objects to floats

In [57]:
final_df = final_df.astype({'worldwide_gross':'float'})# converting objects to floats

In [58]:
final_df['Profit'] = final_df['worldwide_gross'] - final_df['production_budget'] # creating profit columns

In [59]:
final_df['ROI'] = final_df['Profit'] / final_df['production_budget'] * 100

In [60]:
final_df.head()

,release_date,production_budget,domestic_gross,worldwide_gross,Genre,Action,Adventure,Animation,Comedy,Crime,...,Mystery,Music,Romance,SciFi,TV_Movie,Thriller,War,Western,Profit,ROI
10 Cloverfield Lane,"Mar 11, 2016",5000000.0,72082999.0,108286422.0,53,False,False,False,False,False,...,False,False,False,False,False,True,False,False,103286422.0,2065.728440
10 Days in a Madhouse,"Nov 11, 2015",12000000.0,14616.0,14616.0,18,False,False,False,False,False,...,False,False,False,False,False,False,False,False,-11985384.0,-99.878200
12 Strong,"Jan 19, 2018",35000000.0,45819713.0,71118378.0,10752,False,False,False,False,False,...,False,False,False,False,False,False,True,False,36118378.0,103.195366
12 Years a Slave,"Oct 18, 2013",20000000.0,56671993.0,181025343.0,18,False,False,False,False,False,...,False,False,False,False,False,False,False,False,161025343.0,805.126715
127 Hours,"Nov 5, 2010",18000000.0,18335230.0,60217171.0,12,False,True,False,False,False,...,False,False,False,False,False,False,False,False,42217171.0,234.539839


In [61]:
ActAdv_df = final_df.loc[:, ['ROI', 'Profit', 'release_date', 'production_budget', 'domestic_gross', 'worldwide_gross', 
                             "Action","Adventure","War", 'Western']] #creating ActAdv Df

In [62]:
ActAdv_df['ActAdv'] = ActAdv_df["Action"] | ActAdv_df["Adventure"] | ActAdv_df["War"] | ActAdv_df['Western'] #merge columns

In [63]:
ActAdv_df = ActAdv_df.drop('Action', axis=1) #drop redunant columns

In [64]:
ActAdv_df = ActAdv_df.drop('War', axis=1)

In [65]:
ActAdv_df = ActAdv_df.drop('Western', axis=1)

In [66]:
ActAdv_df = ActAdv_df.drop('Adventure', axis=1)

In [67]:
ActAdv_df = ActAdv_df.loc[ActAdv_df.ActAdv, :] #droping all rows not in ActAdv

In [68]:
Drama_df = final_df.loc[:, ['ROI', 'Profit','release_date', 'production_budget', 'domestic_gross', 'worldwide_gross', 
                             'Drama']] #creating Drama Df

In [69]:
Drama_df = Drama_df.loc[Drama_df.Drama, :]

In [70]:
Comedy_df = final_df.loc[:, ['ROI', 'Profit','release_date', 'production_budget', 'domestic_gross', 'worldwide_gross', 
                             'Comedy']] #creating comedy Df

In [71]:
Comedy_df = Comedy_df.loc[Comedy_df.Comedy, :]

In [72]:
Horror_df = final_df.loc[:, ['ROI', 'Profit','release_date', 'production_budget', 'domestic_gross', 'worldwide_gross', 
                             'Horror']] #creating Horror Df

In [73]:
Horror_df = Horror_df.loc[Horror_df.Horror, :]

In [74]:
Thriller_df = final_df.loc[:, ['ROI', 'Profit','release_date', 'production_budget', 'domestic_gross', 'worldwide_gross', 
                             'Thriller', 'Mystery']] #creating thriller Df

In [75]:
Thriller_df['Thrill'] = Thriller_df["Thriller"] | Thriller_df["Mystery"]

In [76]:
Thriller_df = Thriller_df.loc[Thriller_df.Thrill, :]

In [77]:
Thriller_df = Thriller_df.drop('Thriller', axis=1) #drop redunant columns

In [78]:
Thriller_df = Thriller_df.drop('Mystery', axis=1) #drop redunant columns

In [79]:
Crime_df = final_df.loc[:, ['ROI', 'Profit','release_date', 'production_budget', 'domestic_gross', 'worldwide_gross', 
                             'Crime']] #creating Crime Df

In [80]:
Crime_df = Crime_df.loc[Crime_df.Crime, :]

In [81]:
Docu_df = final_df.loc[:, ['ROI', 'Profit','release_date', 'production_budget', 'domestic_gross', 'worldwide_gross', 
                             'Documentary', 'History']] #creating Docu Df

In [82]:
Docu_df['Docu'] = Docu_df["Documentary"] | Docu_df["History"]

In [83]:
Docu_df = Docu_df.drop('Documentary', axis=1) #drop redunant columns

In [84]:
Docu_df = Docu_df.drop('History', axis=1) #drop redunant columns

In [85]:
Docu_df = Docu_df.loc[Docu_df.Docu, :]

In [86]:
Rom_df = final_df.loc[:, ['ROI', 'Profit','release_date', 'production_budget', 'domestic_gross', 'worldwide_gross', 
                             'Romance']] #creating Romance Df

In [87]:
Rom_df = Rom_df.loc[Rom_df.Romance, :]

In [88]:
SciFan_df = final_df.loc[:, ['ROI', 'Profit','release_date', 'production_budget', 'domestic_gross', 'worldwide_gross', 
                             'SciFi', 'Fantasy']] #creating Scifi Fantasy Df

In [89]:
SciFan_df['SciFan'] = SciFan_df["SciFi"] | SciFan_df["Fantasy"]

In [90]:
SciFan_df = SciFan_df.drop('Fantasy', axis=1) #drop redunant columns

In [91]:
SciFan_df = SciFan_df.drop('SciFi', axis=1) #drop redunant columns

In [92]:
SciFan_df = SciFan_df.loc[SciFan_df.SciFan, :]

In [93]:
Fam_df = final_df.loc[:, ['ROI', 'Profit','release_date', 'production_budget', 'domestic_gross', 'worldwide_gross', 
                             'Animation', 'Family', 'Music']] #creating Family Df

In [94]:
Fam_df['Fam'] = Fam_df["Animation"] | Fam_df["Family"] | Fam_df['Music']

In [95]:
Fam_df = Fam_df.drop('Family', axis=1) #drop redunant columns

In [96]:
Fam_df = Fam_df.drop('Music', axis=1) #drop redunant columns

In [97]:
Fam_df = Fam_df.drop('Animation', axis=1) #drop redunant columns

In [98]:
Fam_df = Fam_df.loc[Fam_df.Fam, :]

In [114]:
ActAdv_df['ROI'].describe()

count     548.000000
mean      184.057372
std       341.501361
min       -99.997400
25%       -10.417533
50%       100.461677
75%       263.594136
max      2617.924114
Name: ROI, dtype: float64

In [115]:
Drama_df['ROI'].describe()

count     733.000000
mean      421.262441
std      1353.953233
min       -99.878200
25%       -48.023785
50%        81.037490
75%       306.539022
max      9900.000000
Name: ROI, dtype: float64

In [116]:
Comedy_df['ROI'].describe()

count     407.000000
mean      269.509312
std       750.003623
min       -99.878200
25%       -22.018937
50%       103.641000
75%       253.261420
max      5479.296120
Name: ROI, dtype: float64

In [117]:
Horror_df['ROI'].describe()

count      257.000000
mean      1372.866856
std       3698.182202
min        -99.883429
25%        -27.439268
50%        160.182977
75%       1184.816260
max      41556.474000
Name: ROI, dtype: float64

In [118]:
Thriller_df['ROI'].describe()

count      263.000000
mean       461.020040
std       1605.209264
min        -99.986200
25%        -55.817447
50%         65.420546
75%        366.012800
max      21438.461538
Name: ROI, dtype: float64

In [119]:
Crime_df['ROI'].describe()

count     115.000000
mean      108.434204
std       208.833387
min       -99.335886
25%       -38.284622
50%        69.701800
75%       233.778014
max      1234.932625
Name: ROI, dtype: float64

In [120]:
Docu_df['ROI'].describe()

count     163.000000
mean      171.481186
std       760.848117
min       -99.635600
25%       -73.044292
50%       -40.897943
75%       141.376967
max      6211.912000
Name: ROI, dtype: float64

In [121]:
Rom_df['ROI'].describe()

count      99.000000
mean      863.023300
std      2144.601949
min       -99.479092
25%         0.431861
50%       235.896254
75%       462.685635
max      8989.359138
Name: ROI, dtype: float64

In [122]:
SciFan_df['ROI'].describe()

count     169.000000
mean     1776.223207
std      3198.197799
min       -99.784976
25%        81.235867
50%       196.921458
75%      1079.569540
max      8858.633600
Name: ROI, dtype: float64

In [123]:
Fam_df['ROI'].describe()

count     105.000000
mean      401.822913
std       941.729864
min       -99.836209
25%        37.427040
50%       174.250506
75%       412.745555
max      6211.912000
Name: ROI, dtype: float64

In [101]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2974 entries, 10 Cloverfield Lane to xXx: Return of Xander Cage
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   release_date       2974 non-null   object 
 1   production_budget  2974 non-null   float64
 2   domestic_gross     2974 non-null   float64
 3   worldwide_gross    2974 non-null   float64
 4   Genre              2974 non-null   object 
 5   Action             2974 non-null   bool   
 6   Adventure          2974 non-null   bool   
 7   Animation          2974 non-null   bool   
 8   Comedy             2974 non-null   bool   
 9   Crime              2974 non-null   bool   
 10  Documentary        2974 non-null   bool   
 11  Drama              2974 non-null   bool   
 12  Family             2974 non-null   bool   
 13  Fantasy            2974 non-null   bool   
 14  History            2974 non-null   bool   
 15  Horror             2974 non-null   bo